<a href="https://colab.research.google.com/github/Annareddy-vinay/CodeClauseInternship_Data_Science/blob/main/fake_news_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
#import necessary libraries
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [108]:
#import stop words from nltk
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [109]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [110]:
#load csv file
data=pd.read_csv("/content/train.csv")

In [111]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [112]:
data.tail()

,id,title,author,text,label
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
20799,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [113]:
data.shape

(20800, 5)

In [114]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [115]:
data=data.fillna('')

In [116]:
data["title+author"]=data["title"]+' '+data["author"]

In [117]:
data["title+author"]

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2        Why the Truth Might Get You Fired Consortiumne...
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799              What Keeps the F-35 Alive David Swanson
Name: title+author, Length: 20800, dtype: object

In [118]:
from nltk.stem.porter import  PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [119]:
#finding root words
porter_stemming=PorterStemmer()

In [120]:
def stemming_process(content):
  stem_content=re.sub('[^a-zA-Z]',' ',content)
  stem_content=stem_content.lower()
  stem_content=stem_content.split()
  stem_content=[porter_stemming.stem(word) for word in stem_content if not
                 word in stopwords.words('english')]
  stem_content=' '.join(stem_content)
  return stem_content

In [121]:
data["title+author"]=data["title+author"].apply(stemming_process)

In [122]:
data["title+author"]

0        hous dem aid even see comey letter jason chaff...
1        flynn hillari clinton big woman campu breitbar...
2                   truth might get fire consortiumnew com
3        civilian kill singl us airstrik identifi jessi...
4        iranian woman jail fiction unpublish stori wom...
                               ...                        
20795    rapper trump poster child white supremaci jero...
20796    n f l playoff schedul matchup odd new york tim...
20797    maci said receiv takeov approach hudson bay ne...
20798    nato russia hold parallel exercis balkan alex ...
20799                            keep f aliv david swanson
Name: title+author, Length: 20800, dtype: object

In [123]:
#Devide the input features and output labels
x=data["title+author"]
y=data["label"]

In [124]:
x

0        hous dem aid even see comey letter jason chaff...
1        flynn hillari clinton big woman campu breitbar...
2                   truth might get fire consortiumnew com
3        civilian kill singl us airstrik identifi jessi...
4        iranian woman jail fiction unpublish stori wom...
                               ...                        
20795    rapper trump poster child white supremaci jero...
20796    n f l playoff schedul matchup odd new york tim...
20797    maci said receiv takeov approach hudson bay ne...
20798    nato russia hold parallel exercis balkan alex ...
20799                            keep f aliv david swanson
Name: title+author, Length: 20800, dtype: object

In [125]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

In [126]:
#converting text into numerical
vectorizer=TfidfVectorizer().fit(x)
x=vectorizer.transform(x)

In [127]:
print(x)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [128]:
#splittting for train and test
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y)

In [129]:
print(x_train)

  (0, 16996)	0.10006888917593318
  (0, 11790)	0.27842047026947336
  (0, 10306)	0.09672858590794622
  (0, 6816)	0.17328201706670016
  (0, 6289)	0.3103485948161084
  (0, 5941)	0.3103485948161084
  (0, 4625)	0.1962897341004896
  (0, 4133)	0.4667148238036012
  (0, 3632)	0.22950144583339852
  (0, 3062)	0.36420249980927033
  (0, 2813)	0.17371845705148803
  (0, 681)	0.3229611574758951
  (0, 291)	0.3229611574758951
  (1, 16348)	0.27484531743241825
  (1, 13438)	0.3436921641001472
  (1, 9884)	0.409658978947716
  (1, 9329)	0.3079722524030112
  (1, 7005)	0.28506729566236855
  (1, 6658)	0.32503354017214786
  (1, 2294)	0.34717470521654137
  (1, 1894)	0.1719764708686744
  (1, 469)	0.2752648933096995
  (1, 111)	0.361775139116039
  (2, 15686)	0.4305120325284147
  (2, 15582)	0.17589479373120398
  :	:
  (16636, 1436)	0.30944795736721775
  (16637, 13766)	0.5808711079204887
  (16637, 5238)	0.537078088319386
  (16637, 235)	0.6116664802246546
  (16638, 16217)	0.36575636248905974
  (16638, 13745)	0.2619305351

In [130]:
print(x_test)

  (0, 16868)	0.2761831625109083
  (0, 14444)	0.20003175514203989
  (0, 14389)	0.25486164455285304
  (0, 14037)	0.3259793128964708
  (0, 13460)	0.2857327460886012
  (0, 12181)	0.2767558193518655
  (0, 8565)	0.27451111254121346
  (0, 7044)	0.3630879163469752
  (0, 4928)	0.3547566785615074
  (0, 2184)	0.24885252729193574
  (0, 1331)	0.27035595132193363
  (0, 234)	0.29358949913545246
  (1, 15582)	0.1302356650696384
  (1, 11897)	0.32228762215856355
  (1, 10254)	0.2737866664939124
  (1, 10043)	0.3285414134871024
  (1, 8565)	0.3148429330868978
  (1, 7236)	0.27071355297200744
  (1, 4250)	0.19693933029590432
  (1, 3470)	0.4027351695633435
  (1, 2219)	0.25074983628878356
  (1, 1894)	0.1476720317381719
  (1, 1075)	0.488052901606641
  (2, 520)	1.0
  (3, 17020)	0.3289537883004476
  :	:
  (4157, 1894)	0.1973415686118792
  (4158, 16191)	0.37253491212305634
  (4158, 15019)	0.27433432386168527
  (4158, 14555)	0.6117849378639192
  (4158, 9588)	0.19101324108415657
  (4158, 8485)	0.27271030088568937
  (41

In [131]:
print(y_test)

4993     1
6563     0
2814     1
16803    1
10427    0
        ..
19930    0
15611    1
6329     0
18473    1
17170    0
Name: label, Length: 4160, dtype: int64


In [ ]:
#Fitting and training the model
model=LogisticRegression()
model.fit(x_train,y_train)

In [133]:
#Training accuracy score
x_train_pred=model.predict(x_train)
acc_train=accuracy_score(x_train_pred,y_train)
acc_train

0.9870192307692308

In [134]:
#Testing accuracy score
x_test_pred=model.predict(x_test)
acc_test=accuracy_score(x_test_pred,y_test)
acc_test

0.9771634615384616

In [135]:
# 97.8125% Which is the very good accuracy score

In [136]:
if acc_test<acc_train:
  print("Your model is performing very Well")
else:
  print("Your model is over trained")

Your model is performing very Well


In [140]:
#prediction of values
new=x_test[0]
pred=model.predict(new)
if pred[0]==0:
  print("The news is real")
else:
  print("The news is fake")
print("We can verify by seeing foollowing output")
print("Predicted output is:",pred[0])

The news is fake
We can verify by seeing foollowing output
Predicted output is: 1


In [138]:
y_test

4993     1
6563     0
2814     1
16803    1
10427    0
        ..
19930    0
15611    1
6329     0
18473    1
17170    0
Name: label, Length: 4160, dtype: int64